In [1]:
import numpy as np
import os
from tqdm import tqdm
import sys
from pathlib import Path

from PIL import Image
from datetime import datetime


# Set up paths
NOTEBOOK_DIR = Path.cwd()
SRC_DIR = NOTEBOOK_DIR.parent / "src"
sys.path.append(str(SRC_DIR))

# Import the shared paths
from paths import (
    HOURLY_GREYSCALE_IMAGES_DIR,
    HOURLY_TENSORS_SEASONALITY
)

In [2]:
def get_seasonality_encoding(day_of_year):
    radians = 2 * np.pi * day_of_year / 365.0
    return np.sin(radians), np.cos(radians)

In [21]:
def create_hourly_tensors_from_grayscale(site_name):
    """
    Generate 3D tensors from grayscale PNG heatmaps and add sin/cos encoding of day-of-year.
    Output tensors have shape (79, 168, 3): [grayscale heatmap, sin(doy), cos(doy)]
    """
    input_dir = os.path.join(HOURLY_GREYSCALE_IMAGES_DIR, site_name)
    output_dir = os.path.join(NOTEBOOK_DIR.parent / HOURLY_TENSORS_SEASONALITY, site_name)
    os.makedirs(output_dir, exist_ok=True)

    for filename in tqdm(sorted(os.listdir(input_dir))):
        if not filename.endswith('.png'):
            continue

        try:
            day_str, hour_str = filename.replace('.png', '').split('_hour_')
            date_obj = datetime.strptime(day_str, "%Y-%m-%d")
            day_of_year = date_obj.timetuple().tm_yday
            hour = int(hour_str)
        except Exception as e:
            print(f"Skipping {filename}: bad name format")
            continue

        image_path = os.path.join(input_dir, filename)
        img = Image.open(image_path).convert('L')
        
        # Resize to height=79 and proportionally scale width, 79 because this is the number of altitudes values 
        width_orig, height_orig = img.size
        new_height = 79
        new_width = int((width_orig / height_orig) * new_height)
        img_resized = img.resize((new_width, new_height), Image.BICUBIC)

        img_array = np.array(img_resized).astype(np.float32) / 255.0  # Normalize to [0, 1]
        img_array = np.expand_dims(img_array, axis=-1)  # Shape: (H, W, 1)

        # Pad or crop width, 168 is the max number of profiles included in a single hourly profile 
        target_width = 168
        if img_array.shape[1] < target_width:
            padded = np.zeros((new_height, target_width, 1), dtype=np.float32)
            padded[:, :img_array.shape[1], :] = img_array
            img_array = padded
        else:
            img_array = img_array[:, :target_width, :]

        # Create seasonal feature layers
        sin_doy, cos_doy = get_seasonality_encoding(day_of_year)
        sin_layer = np.full((new_height, target_width, 1), sin_doy, dtype=np.float32)
        cos_layer = np.full((new_height, target_width, 1), cos_doy, dtype=np.float32)

        # Stack all channels
        tensor = np.concatenate([img_array, sin_layer, cos_layer], axis=-1)

        # Save as .npy
        output_path = os.path.join(output_dir, f"{day_str}_hour_{hour:02d}.npy")
        np.save(output_path, tensor)

In [23]:
create_hourly_tensors_from_grayscale(site_name="OT")

100%|████████████████████████████████████████| 823/823 [00:04<00:00, 204.94it/s]


In [25]:
create_hourly_tensors_from_grayscale(site_name="ORM")

100%|██████████████████████████████████████| 1350/1350 [00:06<00:00, 197.73it/s]
